In [ ]:
import matplotlib
import sklearn.metrics
import sklearn.linear_model

import numpy as np
import matplotlib.pyplot as plt
import bootstrapped.bootstrap as bs

%matplotlib inline

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# generate data: part of sine function with random noise
x = np.array([i * np.pi / 180 for i in range(90, 330, 4)])
y = np.sin(x) + np.random.normal(0, 0.2, len(x))

# multiple degrees
X = np.column_stack([x**i for i in range(10)])

plt.plot(x, y, '.')
plt.show()

In [ ]:
ridge = sklearn.linear_model.Ridge(alpha=1e5)
model = ridge.fit(X, y)

def _ridge_predict_error(indexes):
    y_true, y_pred = y[indexes], model.predict(X[indexes, :])
    # return sklearn.metrics.mean_absolute_error(y_true, y_pred)
    return np.percentile(np.abs(y_true - y_pred), q=95)
    
def _ridge_predict_error_bootstrap(indexes):
    if indexes.ndim == 1:
        return [_ridge_predict_error(indexes)]    
    return [_ridge_predict_error(i) for i in indexes]

# do bootstrap to get confidence interval
indexer = np.arange(X.shape[0])
ci = bs.bootstrap(indexer, stat_func=_ridge_predict_error_bootstrap, alpha=0.01)

print("intercept: ", model.intercept_)
print("weights: ", model.coef_)
print("error confidence interval:", (ci.lower_bound, ci.upper_bound))

# prediction
pred = model.predict(X)

plt.plot(x, y, '.')
plt.plot(x, pred)
plt.fill_between(x, pred - ci.lower_bound, pred + ci.lower_bound, color="orange", alpha=0.2)
plt.fill_between(x, pred - ci.upper_bound, pred + ci.upper_bound, color="orange", alpha=0.2)
plt.show()